In [ ]:
#default_exp layers

# Layers
> A bunch of torch layers

In [ ]:
#export
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#export
class ConvLayer(nn.Sequential):
    "A simple Conv+BN+Relu"
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int = 1, groups: int = 1, stride: int = 1, activation: bool = True):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=kernel_size // 2, groups=groups, bias=False),
                  nn.BatchNorm2d(out_channels)] + ([nn.ReLU6(inplace=True)] if activation else [])
        super().__init__(*layers)

In [ ]:
ConvLayer(2,4,1, activation=True)

ConvLayer(
  (0): Conv2d(2, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU6(inplace=True)
)

In [ ]:
ConvLayer(2,4,1, activation=False)

ConvLayer(
  (0): Conv2d(2, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
#export
class CRPBlock(nn.Module):
    "A bunch of convs and a maxpool with a tricky forward"
    def __init__(self, in_channels, out_channels, num_stages=1, use_groups=False):
        super().__init__()
        groups = in_channels if use_groups else 1
        convs = [nn.Conv2d(in_channels if (i == 0) else out_channels, out_channels, kernel_size=1, bias=False, groups=groups) for i in range(num_stages)]
        self.convs = nn.ModuleList(convs)
        self.pool = nn.MaxPool2d(kernel_size=5, stride=1, padding=2)
    
    def forward(self, x):
        "y = x + f(x) + f(f(x)) + f(f(f(x)))..."
        out = x
        for conv in self.convs:
            out = conv(self.pool(out))
            x = out + x
        return x 

In [ ]:
CRPBlock(32,64,3)

CRPBlock(
  (convs): ModuleList(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (pool): MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
)

In [ ]:
#export
class UnetBlock(nn.Module):

    def __init__(self, in_up, in_side, out_channels, kernel_size=1, num_stages=4, use_groups=False):
        super().__init__()
        self.conv_up   = ConvLayer(in_up, out_channels, kernel_size)
        self.conv_side = ConvLayer(in_side, out_channels, kernel_size)
        self.crp = CRPBlock(out_channels, out_channels, num_stages=num_stages, use_groups=use_groups)

    def forward(self, side_input, up_input):
        up_input = self.conv_up(up_input)
        side_input = self.conv_side(side_input)
        if up_input.shape[-2:] != side_input.shape[-2:]:
            up_input = F.interpolate(up_input, size=side_input.shape[-2:], mode="bilinear", align_corners=False)
        out = self.crp(F.relu(up_input + side_input))
        return out

In [ ]:
ublock = UnetBlock(32, 64, 256)

In [ ]:
side = torch.rand(1, 64, 28, 28)
up   = torch.rand(1, 32, 14, 14)
ublock(side, up).shape

torch.Size([1, 256, 28, 28])

In [ ]:
#export
def sigmoid_range(x, low, high):
    "Sigmoid function with range `(low, high)`"
    return torch.sigmoid(x) * (high - low) + low


class SigmoidRange(nn.Module):
    "A simple layer to scale output between `low` and `high`"
    def __init__(self, low, high):
        super().__init__()
        self.low = low
        self.high = high
        
    def forward(self, x):
        "Sigmoid module with range `(low, high)`"
        return sigmoid_range(x, self.low, self.high)

In [ ]:
sr = SigmoidRange(0,10)

In [ ]:
sr(torch.Tensor([-10, -0.1, 1,2,3,4]))

tensor([4.5398e-04, 4.7502e+00, 7.3106e+00, 8.8080e+00, 9.5257e+00, 9.8201e+00])

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data.ipynb.
Converted 02_layers.ipynb.
Converted 03_hydranet.ipynb.
Converted 04_trainer.ipynb.
